In [1]:
import geopandas as gpd
import folium
import fiona
import json

In [2]:
import pandas as pd

df = pd.read_csv('DataSet/DataSet/test_fix.csv', header=None, names=['time', 'id', 'sog', 'lon', 'lat', 'cog', 'delta_time', 'delta_lat', 'delta_long'])

In [27]:
# Convert 'time' column to datetime format
df['time'] = pd.to_datetime(df['time'])

C:\Users\rezas\AppData\Local\Temp\ipykernel_38424\98624784.py:2: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['time'] = pd.to_datetime(df['time'])


In [72]:
df.head(3)

,time,id,sog,lon,lat,cog,delta_time,delta_lat,delta_long
0,25/07/2020 16:28:38,413705370,8.8,122.904993,31.593330,177.8,0,0.000000,0.000000
1,25/07/2020 16:28:48,413705370,8.8,122.905010,31.592925,178.3,10000,0.000017,-0.000405
2,25/07/2020 16:28:59,413705370,8.7,122.905028,31.592520,179.0,11000,0.000018,-0.000405


In [29]:
# convert time to string
df['time'] = df['time'].astype(str)

In [30]:
df.head(3)

,time,id,sog,lon,lat,cog,delta_time,delta_lat,delta_long
0,2020-07-25 15:10:21,414346000,0.1,113.987000,22.024117,297.0,0,0.000000,0.000000
1,2020-07-25 15:10:30,414346000,0.2,113.986983,22.024117,298.0,9000,-0.000017,0.000000
2,2020-07-25 15:10:40,414346000,0.3,113.986967,22.024133,298.0,10000,-0.000016,0.000016


In [25]:
# get the number of unique ships
df['id'].nunique()

57

In [34]:
from keplergl import KeplerGl
# make ipywidgets to display the map
from ipywidgets import interact
import ipywidgets as widgets


# creat a kepler.gl map in a frame
map_1 = KeplerGl(height=400, data={"data_1": df})
# make the center of the map to be the center of the data
center = [df['lat'].mean(), df['lon'].mean()]
map_1.config = {'version': 'v1', 'config': {'mapState': {'latitude': center[0], 'longitude': center[1], 'zoom': 10}}}


# show the map
map_1.save_to_html(file_name="kepler.html")

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to kepler.html!


In [100]:
from geopy.distance import geodesic


# Enable KML support which is disabled by default
fiona.drvsupport.supported_drivers['kml'] = 'rw' # enable KML support which is disabled by default
fiona.drvsupport.supported_drivers['KML'] = 'rw' # enable KML support which is disabled by default
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw' # enable KML support which is disabled by default
# Load KML files using geopandas
path = './Visualization/'
kml_files = ['full_source.kml', 'source.kml', 'true.kml', 'pred_LSTM.kml']  # Add paths to KML files
gdf_list = [gpd.read_file(path + kml) for kml in kml_files]

# Convert GeoDataFrames to GeoJSON strings
gdf_json_list = [gdf.to_crs(epsg='4326').to_json() for gdf in gdf_list]

# Parse the GeoJSON strings
gdf_geojson_list = [json.loads(gdf_json) for gdf_json in gdf_json_list]

# crea a list of colors to use for the layers
colors = ['grey', 'blue', 'red', 'green']

# creat map frame
f = folium.Figure(width=500, height=500)

# Create a folium map with center location and zoom level on the map
map_obj = folium.Map(location=[gdf_geojson_list[0]['features'][0]['geometry']['coordinates'][0][1],
                               gdf_geojson_list[0]['features'][0]['geometry']['coordinates'][0][0]],
                     zoom_start=10).add_to(f)



# check if the geometry is line or point
for i in range(len(gdf_list)):
    if gdf_list[i].geometry.geom_type.unique() == 'LineString':
        # Add GeoJSON layers to the map
        # for i, gdf_geojson in enumerate(gdf_geojson_list):
        gdf_geojson = gdf_geojson_list[i]
        folium.GeoJson(
            gdf_geojson,
            name=kml_files[i].split('.')[0],
            style_function=lambda x, color=colors[i]: {
                'color': color,
                'weight': 2,
                'fillOpacity': 0.5,
            }
            
        ).add_to(map_obj)
    else:
        # Plot points with distance greater than 500 meters from the last plotted point
        prev_point = None
        for point in gdf_list[i].geometry:
            if prev_point:
                distance = geodesic((prev_point.y, prev_point.x), (point.y, point.x)).meters
                if distance > 2500:
                    folium.Circle(location=[point.y, point.x], radius=500, color='red', fill=True, fill_color='red', fill_opacity=1).add_to(map_obj)
                    prev_point = point
            else:
                folium.Circle(location=[point.y, point.x], radius=500, color='red', fill=True, fill_color='red', fill_opacity=1).add_to(map_obj)
                prev_point = point
        


# # Define colors and shapes for markers
# colors = ['red', 'blue', 'green']  # Example colors
# shapes = ['circle', 'square', 'star']  # Example shapes

# # Add points to the map with different colors and shapes
# for i, point in enumerate(points):
#     color = colors[i % len(colors)]
#     shape = shapes[i % len(shapes)]
#     folium.Marker(location=[point.y, point.x], icon=folium.Icon(color=color, icon=shape)).add_to(m)
# # Plot lines
# for line in gdf_list[0].geometry:
#     folium.PolyLine(locations=[(coord[1], coord[0]) for coord in line.coords], color='blue').add_to(map_obj)

# Plot points

# for point in gdf_list[1].geometry:
#     # folium.CircleMarker(location=[point.y, point.x], color='red', fill=True, fill_color='red').add_to(map_obj)
#     folium.Circle(location=[point.y, point.x], radius=100, fill_color="orange", fill_opacity=0.9, weight=1).add_to(map_obj)



    
# Add Layer Control to the map
folium.LayerControl().add_to(map_obj)

# Display the map
map_obj


LineString
